# dataretrieval Overview

## What is dataretrieval?
`dataretrieval` was created to simplify the process of loading hydrologic data into the Python environment.
Like the original R version [`dataRetrieval`](https://github.com/DOI-USGS/dataRetrieval),
it is designed to retrieve the major data types of U.S. Geological Survey (USGS) hydrology
data that are available on the Web, as well as data from the Water
Quality Portal (WQP), which houses water quality data from many organizations and agencies in the U.S. including the
Environmental Protection Agency (EPA) and USGS.

By the end of this tutorial, you should be comfortable using the `dataretrieval` package to query
water data from USGS or the multi-agency Water Quality Portal. The tutorial also demonstrates several simple but versitile patterns for cleaning and visualizing those data.

## R tutorial
For those that prefer R, Laura DeCicco has a nice tutorial [here](https://code.usgs.gov/water/dataRetrieval/-/blob/main/vignettes/tutorial.Rmd?ref_type=heads).
The content diverge at several points but the learning outcomes are similar.

## Installation

`dataretrieval` is available on the Python Package Index (PyPI).
If it is not included in your Python environment, install it from PyPI

```bash
pip install dataretrieval
```
or `conda-forge`
```bash
conda install -c conda-forge dataretrieval
```

## NWIS module
This notebook introduces the `dataretrieval.nwis` module, which contains functions for querying USGS's National Water Information System (NWIS).

### Warning
As of March 2024, dataretrieval will rely on the Water Quality Portal (WQP) services to obtain all discrete water quality data from the USGS. This change was made to accommodate recent changes in the NWIS water quality data web services. Note also that the WQP services are being updated; WQP retrievals will be covered below.

Begin by importing the `dataretrieval.nwis` module.

In [ ]:
from dataretrieval import nwis

The user functions in the `nwis` module are prefixed with `get_`.
The one exception is the  function `what_sites`, but it's just `get_info`

To get a sense of what's available, we can list any function name including the strings 'get' or 'what':

In [ ]:
[i  for i in dir(nwis) if 'get' in i or 'what' in i]

(note: we can view a function's doc using `help(nwis.get_dv)` or `nwis.get_dv?`

In [ ]:
nwis.get_dv?

## National Water Information System (NWIS)  Retrievals
USGS water data is housed in the National Water Information System (NWIS). 
There are many types of data served from NWIS. 
NWIS serves several types of data, but before we dive into these services, it's helpful to understand some codes.

The USGS uses various codes for basic retrievals. These codes can have leading zeros, therefore in Python the data type needs to be define as string ("01234567").

* Site ID (often 8 or 15-digits)
* Parameter Code (5 digits)
    + Full list: [https://nwis.waterdata.usgs.gov/usa/nwis/pmcodes](https://nwis.waterdata.usgs.gov/usa/nwis/pmcodes)
    + Alternatively use `nwis.get_pmcodes("all")`
* Statistic Code (for daily values)
    + Full list: [http://help.waterdata.usgs.gov/code/stat_cd_nm_query?stat_nm_cd=%25&fmt=html](http://help.waterdata.usgs.gov/code/stat_cd_nm_query?stat_nm_cd=%25&fmt=html)

`dataretrieval` queries return data as a `pandas.DataFrame`,
which is convenient for many types of analysis.
However, a dataframes have limited support for metadata.
As a result, `dataretrieval` functions typically return a tuple of two elements: the dataframe and its metadata.

In [ ]:
df, meta = nwis.get_pmcodes("00060"); df

Here a few of the most common parameter codes

| Code  | Name         |
| ------| ------------ |
|00060  | Discharge    |
|00065  | Gage Height  |
|00010	| Temperature  |
|00400	| pH           |

And here are a few common statistic codes

| Code  | Name         |
| ------| ------------ |
|00001  | Maximum      |
|00002  | Minimum      |
|00003	| Mean         |
|00008	| Median       |


The `nwis.get_pmcodes` function returns information on USGS parameter codes. 
You can use "all" to get a full list, then filter out those of interest.

In [ ]:
df, _ = nwis.get_pmcodes("all")

print(f"There are {df.shape[0]} parameter codes")

Here is one example to find all the phosphorous parameter codes:

In [ ]:
df[df.parm_nm.str.contains('Phosphorus', case=False)].head(10)

## Timeseries query


In [ ]:
site = "05427948"
pcode = "00060"
start = "2017-10-01"
end = "2024-05-30"


df, meta = nwis.get_dv(
    sites = site,
    start = start,
    end = end,
    parameterCd = pcode, # kwarg passed to NWIS API
)

Unless your application requires high-frequency (sub daily) data,
I recommend you start with daily values.
- Daily datasets are smaller,faster to download, and consume less memory.
- Instantaneous data commonly have more gaps and require extra processing,
- Missing daily values are estimated and flagged with an 'e'.

From the Pheasant Creek example, let’s look at the data.

In [ ]:
df

In [ ]:
meta # the class _repr_ displays the query url

The metadata records the query url, which can be useful for debugging,
as well as the date and time of the query.

In [ ]:
meta.header

Now let's use `pandas` built-in plotting to visualize our query results.

In [ ]:
df.plot()

Uh oh. If your plot looks strange, try filtering to just the "approved" data

In [ ]:
df.loc[~df['00060_Mean_cd'].str.contains('A')] = None
df.plot()

Better!

When doing some initial exploration, it's much nicer to have interactive plots. 
Python has a diverse plotting ecosystem.
Most libraries will try to give a `matplotlib`-like experience. For example, `pandas` plotting uses `matplotlib` backend by default with many of the same arguments. 
For this reason, Python programmers generally learn `matplotlib` before experimenting with other libraries.
But the other libraries are extremely good too. Here we'll demonstrate `hvplot` for interactive plotting, which has good integration with `pandas` and `xarray`.

In [ ]:
import hvplot.pandas

df.hvplot() # add the letters "hv"

# or
# pd.options.plotting.backend = "holoviews"

#### Here's another common "gotcha":

In [ ]:
from dataretrieval import nwis

df, metadata = nwis.get_dv(site = '08271000', start='1900-01-01')
df.plot()

This record has gaps. To correct it, we populate those missing periods with nan's.
This is fairly easy for single sites:

In [ ]:
# fill the missing data with nans
df.asfreq(freq='D').plot()

But queries returning multiple sites are trickier. In fairness, `pandas.MultiIndex` can make simple tasks tricky in general.

In [ ]:
df, _ = nwis.get_dv(sites=["08271000", "08267500"], parameterCd="00060", start="1901-01-01")
df.head()

For example, how would you go about nan-ing the multi-site dataframe?
Can you find a solution to this "simple" problem?

In [ ]:
# will return an error
df.asfreq(freq='D')

## Query site information
Sometimes we need metadata about a monitoring location.
For this we can use `nwis.get_info` or `nwis.what_sites`,
which are actually the same function with different names.

We can pull metadata for a single location.

In [ ]:
site = "05407000"
df, meta = nwis.what_sites(sites=site)

df

Or construct a multi-location query by providing list of site codes, a state code, or a bounding box.

This query will return all stations with data on a particular parameter in one state.

In [ ]:
state_cd = "IL"
parameter_cd = "00665" # USGS code for total phosphorus
df, meta = nwis.what_sites(
    stateCd=state_cd, 
    parameterCd=parameter_cd,
)

Just as a quick example, let's visualize those sites on an interactive map.

In [ ]:
import geopandas as gpd
import hvplot.pandas

geometry = gpd.points_from_xy(df.dec_long_va, df.dec_lat_va)
gdf = gpd.GeoDataFrame(df, geometry=geometry)

gdf.hvplot.points(
    geo=True,
    hover_cols=["site_no", "station_nm"],
    tiles=True,
    frame_width=300,
    size=4,
)

A typical workflow might begin by downloading data from these sites.
Think 

In [ ]:
df, _ = nwis.get_qwdata(df.site_no.to_list())

That query probably didn't work for you.
In general, `dataretrieval` follows the KISS philosophy:
we won't make up for limitations in the web service; that's up to you.
In this case, we provide an error message with a pseudo-code solution,
whereas `hyriver` would fix this behind the scenes.
But QWData is going the way of the dinosaurs,
and going forward you should be pulling these data from Water Quality Portal (WQP),
which we demonstration in the next [notebook](./01-navigating-wqp.ipynb).

Before moving on to WQP, let's investigate one more service and demonstrate how services can work in conjunction.

Let's query the NWIS "stats" service for a particular streamgage.

In [ ]:
# feel free to "Google" your own gage here: "usgs discharge Illinois River", etc.

df, _ = nwis.get_stats(sites="03339000")

# list what parameters are measured at the gage
df.parameter_cd.unique()

Here we list the parameter codes measured at this site.
More on parameter codes ("p codes") later, but they can be cryptic.
Fortunately, USGS has a web service for that. 
Let's use `nwis.get_pmcodes` to decipher them.

In [ ]:
codes_df, _ = nwis.get_pmcodes(
    df.parameter_cd.unique().tolist()
)

codes_df

That's better. Now select a parameter of interest, and we'll quickly plot it.

In [ ]:
parameter = "99133" # nitrate
parm_df = df[df["parameter_cd"] == parameter].reset_index(drop=True)
parm_df

In [ ]:
parm_df[["mean_va","p20_va","p80_va"]].plot()

Try plotting a different parameter before we move on to the next notebook.